## Importing Stock Data with Yahoo Finance

- In this notebook you will find a Python script that provides a stock dataset that comes from Yahoo Finance. This dataset will be used for analysis with PowerBI.

### 1. Importing Librarys

In [ ]:
import pandas as pd
import yfinance as yf

### 2. Setting some parameters:

In [ ]:
stocks = ['F', 'AAPL', 'NVDA', 'TSLA', 'AMZN', 'MSFT', 'GOOGL', 'META', 'NFLX', 'AMD']
start_date = '2015-05-01'
end_date = '2025-05-01'

### 3. Making a Stock Dataset:

In [ ]:
for i in range(len(stocks)):
    database = yf.download(stocks[i], start=start_date, end=end_date, progress=False)
    database = database.reset_index()
    database.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    database['Symbol Stock'] = stocks[i]

    if i == 0:
        aux = database.copy()
        pass
    else:
        database = pd.concat([database, aux], axis=0)
        aux = database.copy()



In [115]:
database.groupby('Symbol Stock').size()

Symbol Stock
AAPL     2515
AMD      2515
AMZN     2515
F        2515
GOOGL    2515
META     2515
MSFT     2515
NFLX     2515
NVDA     2515
TSLA     2515
dtype: int64

### 4. Importing the Dataset into a "csv" file:

In [116]:
database.to_csv('stock-analysis_2015-2025.csv', index=False)